# HyperCLOVAX Batch Size별 파인튜닝 성능 비교: 한국어 텍스트 비난독화

이 노트북은 Naver HyperCLOVAX-SEED-Text-Instruct-0.5B 모델을 한국어 문자열 비난독화를 위해 **다양한 배치 사이즈**로 fine-tuning하는 실험을 수행합니다.

## 🎯 실험 목표
- 난독화된 한국어 텍스트를 원본 텍스트로 복원하는 모델 훈련
- **배치 사이즈 (1, 2, 4) 변화에 따른 성능 비교 분석**
- LoRA (Low-Rank Adaptation)를 사용한 효율적인 fine-tuning
- 다양한 텍스트 유형에 대한 성능 평가 (구어체, 뉴스, 문화, 전문분야, 조례, 지자체웹사이트)

## 📋 환경 및 경로 설정

이 노트북은 **Google Colab** 환경에서 실행되도록 최적화되어 있습니다.

### 💾 파일 구조
- **데이터 파일**: Google Drive의 MyDrive 루트에 위치
- **모델 저장**: Google Drive의 `hyperclova-deobfuscation-lora-batch{size}/` 폴더
- **결과 파일**: 현재 작업 디렉터리에 저장 후 다운로드

### 🔧 사전 준비사항
1. Google Drive에 다음 데이터 파일들이 업로드되어 있어야 합니다:
   - `구어체_대화체_16878_sample_난독화결과.csv`
   - `뉴스문어체_281932_sample_난독화결과.csv`
   - `문화문어체_25628_sample_난독화결과.csv`
   - `전문분야 문어체_306542_sample_난독화결과.csv`
   - `조례문어체_36339_sample_난독화결과.csv`
   - `지자체웹사이트 문어체_28705_sample_난독화결과.csv`

2. Google Drive 접근 권한 허용이 필요합니다.

### 🧪 실험 설계
이 노트북에서는 다음 배치 사이즈로 각각 모델을 훈련하고 성능을 비교합니다:
- **Batch Size 1**: 메모리 효율적, 학습 안정성 높음
- **Batch Size 2**: 균형잡힌 설정
- **Batch Size 4**: 빠른 수렴, 높은 메모리 사용

---

## 1. 환경 설정 및 라이브러리 설치

In [ ]:
# GPU 확인
print("=== GPU 정보 확인 ===")
!nvidia-smi

print("\n=== 필수 패키지 설치 ===")
print("패키지 설치 중... (약 3-5분 소요)")

# 필요한 패키지 설치
!pip install -q transformers>=4.35.0
!pip install -q peft>=0.6.0
!pip install -q trl>=0.7.0
!pip install -q datasets>=2.14.0
!pip install -q bitsandbytes>=0.41.0
!pip install -q accelerate>=0.24.0
!pip install -q evaluate>=0.4.0
!pip install -q rouge-score>=0.1.2
!pip install -q sentencepiece>=0.1.99
!pip install -q protobuf>=3.20.0
!pip install -q gradio>=4.0.0
!pip install -q scikit-learn>=1.3.0

print("패키지 설치 완료!")

# Hugging Face Hub 로그인 (필요시)
print("\n=== Hugging Face 로그인 ===")
from huggingface_hub import notebook_login
print("비공개 모델을 사용하는 경우 아래 주석을 해제하여 로그인하세요:")
# notebook_login()  # 필요한 경우 주석 해제

In [ ]:
import torch
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    prepare_model_for_kbit_training
)
from trl import SFTTrainer
import warnings
warnings.filterwarnings('ignore')

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 디바이스: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 2. 데이터 로딩 및 전처리

In [ ]:
# Google Drive 연결 및 BASE_PATH 설정
from google.colab import drive
drive.mount('/content/drive')

# 환경별 경로 설정
BASE_PATH = '/content/drive/MyDrive/'
print(f"기본 경로 설정: {BASE_PATH}")

# 또는 직접 파일 업로드
from google.colab import files
# uploaded = files.upload()  # 필요시 주석 해제

In [ ]:
# 데이터셋 파일 경로 설정 (BASE_PATH 사용)
data_files = {
    '구어체_대화체': BASE_PATH + '구어체_대화체_16878_sample_난독화결과.csv',
    '뉴스문어체': BASE_PATH + '뉴스문어체_281932_sample_난독화결과.csv',
    '문화문어체': BASE_PATH + '문화문어체_25628_sample_난독화결과.csv',
    '전문분야문어체': BASE_PATH + '전문분야 문어체_306542_sample_난독화결과.csv',
    '조례문어체': BASE_PATH + '조례문어체_36339_sample_난독화결과.csv',
    '지자체웹사이트문어체': BASE_PATH + '지자체웹사이트 문어체_28705_sample_난독화결과.csv'
}

print("데이터 파일 경로 목록:")
for name, path in data_files.items():
    print(f"  {name}: {path}")

# 모든 데이터셋 로드 및 결합
all_data = []
for name, file_path in data_files.items():
    try:
        df = pd.read_csv(file_path)
        df['category'] = name
        all_data.append(df)
        print(f"{name}: {len(df)} 샘플 로드됨")
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        continue
    except Exception as e:
        print(f"파일 로드 오류 ({name}): {str(e)}")
        continue

# 데이터 결합
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"\n전체 데이터셋 크기: {len(combined_df)}")
    print(f"카테고리별 분포:")
    print(combined_df['category'].value_counts())
else:
    print("\n로드된 데이터가 없습니다. 파일 경로를 확인해주세요.")

In [ ]:
# 데이터 전처리 함수
def create_instruction_dataset(df, sample_size=None):
    """난독화 해제 작업을 위한 instruction 형태 데이터셋 생성"""

    if sample_size:
        df = df.sample(n=sample_size, random_state=42).reset_index(drop=True)

    instructions = []
    for _, row in df.iterrows():
        # Instruction 형태로 변환
        instruction = {
            'input': f"다음 난독화된 한국어 텍스트를 원래 텍스트로 복원해주세요.\n\n난독화된 텍스트: {row['obfuscated']}",
            'output': row['original'],
            'category': row['category']
        }
        instructions.append(instruction)

    return pd.DataFrame(instructions)

# Instruction 데이터셋 생성 (메모리 절약을 위해 샘플링)
sample_size = 10000  # 필요에 따라 조정
instruction_df = create_instruction_dataset(combined_df, sample_size)

print(f"Instruction 데이터셋 크기: {len(instruction_df)}")
print("\n첫 번째 예시:")
print(f"Input: {instruction_df.iloc[0]['input']}")
print(f"Output: {instruction_df.iloc[0]['output']}")

In [ ]:
# 훈련/검증 데이터 분할
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    instruction_df,
    test_size=0.1,
    random_state=42,
    stratify=instruction_df['category']
)

print(f"훈련 데이터: {len(train_df)}")
print(f"검증 데이터: {len(val_df)}")

# Hugging Face Dataset 형태로 변환
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

print("\n데이터셋 구조:")
print(dataset)

## 3. 모델 및 토크나이저 설정

In [ ]:
# 모델 설정
model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B"

# 양자화 설정 (메모리 절약)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(model_name)

# LoRA를 위한 모델 준비
model = prepare_model_for_kbit_training(model)

print(f"모델 로드 완료: {model_name}")
print(f"토크나이저 어휘 크기: {len(tokenizer)}")

In [ ]:
# LoRA 설정
lora_config = LoraConfig(
    r=16,  # rank
    lora_alpha=32,  # scaling parameter
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# LoRA 모델 생성
model = get_peft_model(model, lora_config)

# 훈련 가능한 파라미터 확인
model.print_trainable_parameters()

## 4. 데이터 포맷팅

In [ ]:
# 데이터 포맷팅 함수
def format_instruction(example):
    """Instruction 포맷으로 데이터 변환"""
    prompt = f"### 지시사항:\n{example['input']}\n\n### 응답:\n{example['output']}<|endoftext|>"
    return {"text": prompt}

# 데이터셋에 포맷 적용
formatted_dataset = dataset.map(format_instruction, remove_columns=dataset['train'].column_names)

print("포맷된 데이터 예시:")
print(formatted_dataset['train'][0]['text'][:500] + "...")

## 5. 배치 사이즈별 모델 훈련

In [ ]:
# 배치 사이즈별 실험 설정
BATCH_SIZES = [1, 2, 4]
RESULTS = {}

print("=== 배치 사이즈별 실험 시작 ===")
print(f"실험할 배치 사이즈: {BATCH_SIZES}")

for batch_size in BATCH_SIZES:
    print(f"\n{'='*50}")
    print(f"배치 사이즈 {batch_size} 실험 시작")
    print(f"{'='*50}")
    
    # 모델 저장 경로 설정 (배치 사이즈별)
    MODEL_OUTPUT_DIR = BASE_PATH + f"hyperclova-deobfuscation-lora-batch{batch_size}"
    print(f"모델 저장 경로: {MODEL_OUTPUT_DIR}")

    # 훈련 설정 (배치 사이즈별로 다른 설정)
    training_args = TrainingArguments(
        output_dir=MODEL_OUTPUT_DIR,
        num_train_epochs=3,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=max(1, 4 // batch_size),  # 총 effective batch size 유지
        warmup_steps=100,
        learning_rate=2e-4,
        weight_decay=0.01,
        logging_steps=10,
        eval_strategy="steps",
        eval_steps=200,
        save_steps=200,
        save_total_limit=3,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        fp16=True,
        dataloader_pin_memory=False,
        remove_unused_columns=False,
        report_to="none",
        run_name=f"batch_size_{batch_size}"
    )

    print(f"훈련 설정 완료 - Batch Size: {batch_size}")
    print(f"Gradient Accumulation Steps: {training_args.gradient_accumulation_steps}")
    
    # 모델 재초기화 (각 실험마다 새로운 모델 사용)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)
    
    # SFT Trainer 설정
    trainer = SFTTrainer(
        model=model,
        train_dataset=formatted_dataset['train'],
        eval_dataset=formatted_dataset['validation'],
        args=training_args
    )

    print(f"SFT Trainer 설정 완료 - Batch Size: {batch_size}")
    
    # 훈련 시작
    import time
    start_time = time.time()
    
    print(f"배치 사이즈 {batch_size} 훈련을 시작합니다...")
    train_result = trainer.train()
    
    end_time = time.time()
    training_time = end_time - start_time
    
    # 모델 저장
    trainer.save_model()
    tokenizer.save_pretrained(MODEL_OUTPUT_DIR)
    
    # 결과 저장
    RESULTS[batch_size] = {
        'train_loss': train_result.training_loss,
        'training_time': training_time,
        'model_path': MODEL_OUTPUT_DIR
    }
    
    print(f"배치 사이즈 {batch_size} 훈련 완료!")
    print(f"훈련 시간: {training_time:.2f}초 ({training_time/60:.1f}분)")
    print(f"최종 손실: {train_result.training_loss:.4f}")
    print(f"모델 저장 경로: {MODEL_OUTPUT_DIR}")

print(f"\n{'='*50}")
print("모든 배치 사이즈 실험 완료!")
print(f"{'='*50}")

In [ ]:
# 배치 사이즈별 결과 요약
print("=== 배치 사이즈별 실험 결과 요약 ===")
print("┌─────────────┬──────────────┬──────────────┬─────────────────┐")
print("│ Batch Size  │ Train Loss   │ Time (min)   │ Model Path      │")
print("├─────────────┼──────────────┼──────────────┼─────────────────┤")

for batch_size, result in RESULTS.items():
    train_loss = result['train_loss']
    time_min = result['training_time'] / 60
    model_path = result['model_path'].split('/')[-1]  # 경로의 마지막 부분만
    print(f"│     {batch_size:2d}      │   {train_loss:8.4f}   │   {time_min:8.1f}   │ {model_path:15s} │")

print("└─────────────┴──────────────┴──────────────┴─────────────────┘")

# 최적 배치 사이즈 선정
best_batch_size = min(RESULTS.keys(), key=lambda x: RESULTS[x]['train_loss'])
print(f"\n🏆 최적 배치 사이즈: {best_batch_size} (최저 손실: {RESULTS[best_batch_size]['train_loss']:.4f})")

## 6. 모델 평가 및 테스트

In [ ]:
# 추론 함수 정의
def generate_deobfuscated_text(obfuscated_text, max_length=256):
    """난독화된 텍스트를 입력받아 원본 텍스트 생성"""
    prompt = f"### 지시사항:\n다음 난독화된 한국어 텍스트를 원래 텍스트로 복원해주세요.\n\n난독화된 텍스트: {obfuscated_text}\n\n### 응답:\n"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 응답 부분만 추출
    if "### 응답:" in response:
        response = response.split("### 응답:")[1].strip()

    return response

# 평가 메트릭 함수 정의
def calculate_metrics(predictions, references):
    """BLEU, ROUGE 스코어 계산"""
    from evaluate import load
    
    # 메트릭 로드
    bleu = load("bleu")
    rouge = load("rouge")
    
    # BLEU 계산
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

    # ROUGE 계산
    rouge_score = rouge.compute(predictions=predictions, references=references)

    return {
        'bleu': bleu_score['bleu'],
        'rouge1': rouge_score['rouge1'],
        'rouge2': rouge_score['rouge2'],
        'rougeL': rouge_score['rougeL']
    }

print("추론 및 평가 함수 정의 완료")

In [ ]:
# 각 배치 사이즈별 모델 테스트
print("=== 배치 사이즈별 모델 성능 테스트 ===")

test_samples = val_df.sample(n=5, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n{'='*60}")
    print(f"배치 사이즈 {batch_size} 모델 테스트")
    print(f"{'='*60}")
    
    # 해당 배치 사이즈의 모델 로드
    model_path = RESULTS[batch_size]['model_path']
    
    # 베이스 모델 로드
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # LoRA 어댑터 적용
    from peft import PeftModel
    model = PeftModel.from_pretrained(base_model, model_path)
    
    # 테스트 샘플 평가
    for idx, row in test_samples.iterrows():
        obfuscated = row['input'].split("난독화된 텍스트: ")[1]
        original = row['output']
        predicted = generate_deobfuscated_text(obfuscated)

        print(f"\n[샘플 {idx}] 카테고리: {row['category']}")
        print(f"난독화: {obfuscated}")
        print(f"원본: {original}")
        print(f"예측: {predicted}")
        print("-" * 40)
    
    # 메모리 정리
    del model, base_model
    torch.cuda.empty_cache()

In [ ]:
# 배치 사이즈별 정량적 평가
batch_metrics = {}

print("=== 배치 사이즈별 정량적 평가 ===")

# 평가용 샘플 (50개로 축소)
eval_samples = val_df.sample(n=50, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n배치 사이즈 {batch_size} 평가 중...")
    
    # 모델 로드
    model_path = RESULTS[batch_size]['model_path']
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    model = PeftModel.from_pretrained(base_model, model_path)
    
    predictions = []
    references = []
    
    for idx, row in eval_samples.iterrows():
        obfuscated = row['input'].split("난독화된 텍스트: ")[1]
        original = row['output']
        predicted = generate_deobfuscated_text(obfuscated)
        
        predictions.append(predicted)
        references.append(original)
    
    # 메트릭 계산
    metrics = calculate_metrics(predictions, references)
    batch_metrics[batch_size] = metrics
    
    print(f"BLEU Score: {metrics['bleu']:.4f}")
    print(f"ROUGE-1: {metrics['rouge1']:.4f}")
    print(f"ROUGE-2: {metrics['rouge2']:.4f}")
    print(f"ROUGE-L: {metrics['rougeL']:.4f}")
    
    # 메모리 정리
    del model, base_model
    torch.cuda.empty_cache()

# 결과 비교표
print("\n=== 배치 사이즈별 성능 비교 ===")
print("┌─────────────┬──────────┬──────────┬──────────┬──────────┐")
print("│ Batch Size  │   BLEU   │ ROUGE-1  │ ROUGE-2  │ ROUGE-L  │")
print("├─────────────┼──────────┼──────────┼──────────┼──────────┤")

for batch_size in BATCH_SIZES:
    metrics = batch_metrics[batch_size]
    print(f"│     {batch_size:2d}      │  {metrics['bleu']:6.4f}  │  {metrics['rouge1']:6.4f}  │  {metrics['rouge2']:6.4f}  │  {metrics['rougeL']:6.4f}  │")

print("└─────────────┴──────────┴──────────┴──────────┴──────────┘")

# 최고 성능 배치 사이즈
best_bleu_batch = max(batch_metrics.keys(), key=lambda x: batch_metrics[x]['bleu'])
best_rouge_batch = max(batch_metrics.keys(), key=lambda x: batch_metrics[x]['rougeL'])

print(f"\n🏆 최고 BLEU 성능: 배치 사이즈 {best_bleu_batch} (BLEU: {batch_metrics[best_bleu_batch]['bleu']:.4f})")
print(f"🏆 최고 ROUGE-L 성능: 배치 사이즈 {best_rouge_batch} (ROUGE-L: {batch_metrics[best_rouge_batch]['rougeL']:.4f})")

## 6. 모델 평가 및 테스트

In [ ]:
# 추론 함수 정의
def generate_deobfuscated_text(obfuscated_text, max_length=256):
    """난독화된 텍스트를 입력받아 원본 텍스트 생성"""
    prompt = f"### 지시사항:\n다음 난독화된 한국어 텍스트를 원래 텍스트로 복원해주세요.\n\n난독화된 텍스트: {obfuscated_text}\n\n### 응답:\n"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 응답 부분만 추출
    if "### 응답:" in response:
        response = response.split("### 응답:")[1].strip()

    return response

# 평가 메트릭 함수 정의
def calculate_metrics(predictions, references):
    """BLEU, ROUGE 스코어 계산"""
    from evaluate import load
    
    # 메트릭 로드
    bleu = load("bleu")
    rouge = load("rouge")
    
    # BLEU 계산
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

    # ROUGE 계산
    rouge_score = rouge.compute(predictions=predictions, references=references)

    return {
        'bleu': bleu_score['bleu'],
        'rouge1': rouge_score['rouge1'],
        'rouge2': rouge_score['rouge2'],
        'rougeL': rouge_score['rougeL']
    }

print("추론 및 평가 함수 정의 완료")

In [ ]:
# 각 배치 사이즈별 모델 테스트
print("=== 배치 사이즈별 모델 성능 테스트 ===")

test_samples = val_df.sample(n=5, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n{'='*60}")
    print(f"배치 사이즈 {batch_size} 모델 테스트")
    print(f"{'='*60}")
    
    # 해당 배치 사이즈의 모델 로드
    model_path = RESULTS[batch_size]['model_path']
    
    # 베이스 모델 로드
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # LoRA 어댑터 적용
    from peft import PeftModel
    model = PeftModel.from_pretrained(base_model, model_path)
    
    # 테스트 샘플 평가
    for idx, row in test_samples.iterrows():
        obfuscated = row['input'].split("난독화된 텍스트: ")[1]
        original = row['output']
        predicted = generate_deobfuscated_text(obfuscated)

        print(f"\n[샘플 {idx}] 카테고리: {row['category']}")
        print(f"난독화: {obfuscated}")
        print(f"원본: {original}")
        print(f"예측: {predicted}")
        print("-" * 40)
    
    # 메모리 정리
    del model, base_model
    torch.cuda.empty_cache()

In [ ]:
# 배치 사이즈별 정량적 평가
batch_metrics = {}

print("=== 배치 사이즈별 정량적 평가 ===")

# 평가용 샘플 (50개로 축소)
eval_samples = val_df.sample(n=50, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n배치 사이즈 {batch_size} 평가 중...")
    
    # 모델 로드
    model_path = RESULTS[batch_size]['model_path']
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    model = PeftModel.from_pretrained(base_model, model_path)
    
    predictions = []
    references = []
    
    for idx, row in eval_samples.iterrows():
        obfuscated = row['input'].split("난독화된 텍스트: ")[1]
        original = row['output']
        predicted = generate_deobfuscated_text(obfuscated)
        
        predictions.append(predicted)
        references.append(original)
    
    # 메트릭 계산
    metrics = calculate_metrics(predictions, references)
    batch_metrics[batch_size] = metrics
    
    print(f"BLEU Score: {metrics['bleu']:.4f}")
    print(f"ROUGE-1: {metrics['rouge1']:.4f}")
    print(f"ROUGE-2: {metrics['rouge2']:.4f}")
    print(f"ROUGE-L: {metrics['rougeL']:.4f}")
    
    # 메모리 정리
    del model, base_model
    torch.cuda.empty_cache()

# 결과 비교표
print("\n=== 배치 사이즈별 성능 비교 ===")
print("┌─────────────┬──────────┬──────────┬──────────┬──────────┐")
print("│ Batch Size  │   BLEU   │ ROUGE-1  │ ROUGE-2  │ ROUGE-L  │")
print("├─────────────┼──────────┼──────────┼──────────┼──────────┤")

for batch_size in BATCH_SIZES:
    metrics = batch_metrics[batch_size]
    print(f"│     {batch_size:2d}      │  {metrics['bleu']:6.4f}  │  {metrics['rouge1']:6.4f}  │  {metrics['rouge2']:6.4f}  │  {metrics['rougeL']:6.4f}  │")

print("└─────────────┴──────────┴──────────┴──────────┴──────────┘")

# 최고 성능 배치 사이즈
best_bleu_batch = max(batch_metrics.keys(), key=lambda x: batch_metrics[x]['bleu'])
best_rouge_batch = max(batch_metrics.keys(), key=lambda x: batch_metrics[x]['rougeL'])

print(f"\n🏆 최고 BLEU 성능: 배치 사이즈 {best_bleu_batch} (BLEU: {batch_metrics[best_bleu_batch]['bleu']:.4f})")
print(f"🏆 최고 ROUGE-L 성능: 배치 사이즈 {best_rouge_batch} (ROUGE-L: {batch_metrics[best_rouge_batch]['rougeL']:.4f})")

## 6. 모델 평가 및 테스트

In [ ]:
# 추론 함수 정의
def generate_deobfuscated_text(obfuscated_text, max_length=256):
    """난독화된 텍스트를 입력받아 원본 텍스트 생성"""
    prompt = f"### 지시사항:\n다음 난독화된 한국어 텍스트를 원래 텍스트로 복원해주세요.\n\n난독화된 텍스트: {obfuscated_text}\n\n### 응답:\n"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 응답 부분만 추출
    if "### 응답:" in response:
        response = response.split("### 응답:")[1].strip()

    return response

# 평가 메트릭 함수 정의
def calculate_metrics(predictions, references):
    """BLEU, ROUGE 스코어 계산"""
    from evaluate import load
    
    # 메트릭 로드
    bleu = load("bleu")
    rouge = load("rouge")
    
    # BLEU 계산
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

    # ROUGE 계산
    rouge_score = rouge.compute(predictions=predictions, references=references)

    return {
        'bleu': bleu_score['bleu'],
        'rouge1': rouge_score['rouge1'],
        'rouge2': rouge_score['rouge2'],
        'rougeL': rouge_score['rougeL']
    }

print("추론 및 평가 함수 정의 완료")

In [ ]:
# 각 배치 사이즈별 모델 테스트
print("=== 배치 사이즈별 모델 성능 테스트 ===")

test_samples = val_df.sample(n=5, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n{'='*60}")
    print(f"배치 사이즈 {batch_size} 모델 테스트")
    print(f"{'='*60}")
    
    # 해당 배치 사이즈의 모델 로드
    model_path = RESULTS[batch_size]['model_path']
    
    # 베이스 모델 로드
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # LoRA 어댑터 적용
    from peft import PeftModel
    model = PeftModel.from_pretrained(base_model, model_path)
    
    # 테스트 샘플 평가
    for idx, row in test_samples.iterrows():
        obfuscated = row['input'].split("난독화된 텍스트: ")[1]
        original = row['output']
        predicted = generate_deobfuscated_text(obfuscated)

        print(f"\n[샘플 {idx}] 카테고리: {row['category']}")
        print(f"난독화: {obfuscated}")
        print(f"원본: {original}")
        print(f"예측: {predicted}")
        print("-" * 40)
    
    # 메모리 정리
    del model, base_model
    torch.cuda.empty_cache()

In [ ]:
# 배치 사이즈별 정량적 평가
batch_metrics = {}

print("=== 배치 사이즈별 정량적 평가 ===")

# 평가용 샘플 (50개로 축소)
eval_samples = val_df.sample(n=50, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n배치 사이즈 {batch_size} 평가 중...")
    
    # 모델 로드
    model_path = RESULTS[batch_size]['model_path']
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    model = PeftModel.from_pretrained(base_model, model_path)
    
    predictions = []
    references = []
    
    for idx, row in eval_samples.iterrows():
        obfuscated = row['input'].split("난독화된 텍스트: ")[1]
        original = row['output']
        predicted = generate_deobfuscated_text(obfuscated)
        
        predictions.append(predicted)
        references.append(original)
    
    # 메트릭 계산
    metrics = calculate_metrics(predictions, references)
    batch_metrics[batch_size] = metrics
    
    print(f"BLEU Score: {metrics['bleu']:.4f}")
    print(f"ROUGE-1: {metrics['rouge1']:.4f}")
    print(f"ROUGE-2: {metrics['rouge2']:.4f}")
    print(f"ROUGE-L: {metrics['rougeL']:.4f}")
    
    # 메모리 정리
    del model, base_model
    torch.cuda.empty_cache()

# 결과 비교표
print("\n=== 배치 사이즈별 성능 비교 ===")
print("┌─────────────┬──────────┬──────────┬──────────┬──────────┐")
print("│ Batch Size  │   BLEU   │ ROUGE-1  │ ROUGE-2  │ ROUGE-L  │")
print("├─────────────┼──────────┼──────────┼──────────┼──────────┤")

for batch_size in BATCH_SIZES:
    metrics = batch_metrics[batch_size]
    print(f"│     {batch_size:2d}      │  {metrics['bleu']:6.4f}  │  {metrics['rouge1']:6.4f}  │  {metrics['rouge2']:6.4f}  │  {metrics['rougeL']:6.4f}  │")

print("└─────────────┴──────────┴──────────┴──────────┴──────────┘")

# 최고 성능 배치 사이즈
best_bleu_batch = max(batch_metrics.keys(), key=lambda x: batch_metrics[x]['bleu'])
best_rouge_batch = max(batch_metrics.keys(), key=lambda x: batch_metrics[x]['rougeL'])

print(f"\n🏆 최고 BLEU 성능: 배치 사이즈 {best_bleu_batch} (BLEU: {batch_metrics[best_bleu_batch]['bleu']:.4f})")
print(f"🏆 최고 ROUGE-L 성능: 배치 사이즈 {best_rouge_batch} (ROUGE-L: {batch_metrics[best_rouge_batch]['rougeL']:.4f})")

## 6. 모델 평가 및 테스트

In [ ]:
# 추론 함수 정의
def generate_deobfuscated_text(obfuscated_text, max_length=256):
    """난독화된 텍스트를 입력받아 원본 텍스트 생성"""
    prompt = f"### 지시사항:\n다음 난독화된 한국어 텍스트를 원래 텍스트로 복원해주세요.\n\n난독화된 텍스트: {obfuscated_text}\n\n### 응답:\n"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 응답 부분만 추출
    if "### 응답:" in response:
        response = response.split("### 응답:")[1].strip()

    return response

# 평가 메트릭 함수 정의
def calculate_metrics(predictions, references):
    """BLEU, ROUGE 스코어 계산"""
    from evaluate import load
    
    # 메트릭 로드
    bleu = load("bleu")
    rouge = load("rouge")
    
    # BLEU 계산
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

    # ROUGE 계산
    rouge_score = rouge.compute(predictions=predictions, references=references)

    return {
        'bleu': bleu_score['bleu'],
        'rouge1': rouge_score['rouge1'],
        'rouge2': rouge_score['rouge2'],
        'rougeL': rouge_score['rougeL']
    }

print("추론 및 평가 함수 정의 완료")

In [ ]:
# 각 배치 사이즈별 모델 테스트
print("=== 배치 사이즈별 모델 성능 테스트 ===")

test_samples = val_df.sample(n=5, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n{'='*60}")
    print(f"배치 사이즈 {batch_size} 모델 테스트")
    print(f"{'='*60}")
    
    # 해당 배치 사이즈의 모델 로드
    model_path = RESULTS[batch_size]['model_path']
    
    # 베이스 모델 로드
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # LoRA 어댑터 적용
    from peft import PeftModel
    model = PeftModel.from_pretrained(base_model, model_path)
    
    # 테스트 샘플 평가
    for idx, row in test_samples.iterrows():
        obfuscated = row['input'].split("난독화된 텍스트: ")[1]
        original = row['output']
        predicted = generate_deobfuscated_text(obfuscated)

        print(f"\n[샘플 {idx}] 카테고리: {row['category']}")
        print(f"난독화: {obfuscated}")
        print(f"원본: {original}")
        print(f"예측: {predicted}")
        print("-" * 40)
    
    # 메모리 정리
    del model, base_model
    torch.cuda.empty_cache()

In [ ]:
# 배치 사이즈별 정량적 평가
batch_metrics = {}

print("=== 배치 사이즈별 정량적 평가 ===")

# 평가용 샘플 (50개로 축소)
eval_samples = val_df.sample(n=50, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n배치 사이즈 {batch_size} 평가 중...")
    
    # 모델 로드
    model_path = RESULTS[batch_size]['model_path']
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    model = PeftModel.from_pretrained(base_model, model_path)
    
    predictions = []
    references = []
    
    for idx, row in eval_samples.iterrows():
        obfuscated = row['input'].split("난독화된 텍스트: ")[1]
        original = row['output']
        predicted = generate_deobfuscated_text(obfuscated)
        
        predictions.append(predicted)
        references.append(original)
    
    # 메트릭 계산
    metrics = calculate_metrics(predictions, references)
    batch_metrics[batch_size] = metrics
    
    print(f"BLEU Score: {metrics['bleu']:.4f}")
    print(f"ROUGE-1: {metrics['rouge1']:.4f}")
    print(f"ROUGE-2: {metrics['rouge2']:.4f}")
    print(f"ROUGE-L: {metrics['rougeL']:.4f}")
    
    # 메모리 정리
    del model, base_model
    torch.cuda.empty_cache()

# 결과 비교표
print("\n=== 배치 사이즈별 성능 비교 ===")
print("┌─────────────┬──────────┬──────────┬──────────┬──────────┐")
print("│ Batch Size  │   BLEU   │ ROUGE-1  │ ROUGE-2  │ ROUGE-L  │")
print("├─────────────┼──────────┼──────────┼──────────┼──────────┤")

for batch_size in BATCH_SIZES:
    metrics = batch_metrics[batch_size]
    print(f"│     {batch_size:2d}      │  {metrics['bleu']:6.4f}  │  {metrics['rouge1']:6.4f}  │  {metrics['rouge2']:6.4f}  │  {metrics['rougeL']:6.4f}  │")

print("└─────────────┴──────────┴──────────┴──────────┴──────────┘")

# 최고 성능 배치 사이즈
best_bleu_batch = max(batch_metrics.keys(), key=lambda x: batch_metrics[x]['bleu'])
best_rouge_batch = max(batch_metrics.keys(), key=lambda x: batch_metrics[x]['rougeL'])

print(f"\n🏆 최고 BLEU 성능: 배치 사이즈 {best_bleu_batch} (BLEU: {batch_metrics[best_bleu_batch]['bleu']:.4f})")
print(f"🏆 최고 ROUGE-L 성능: 배치 사이즈 {best_rouge_batch} (ROUGE-L: {batch_metrics[best_rouge_batch]['rougeL']:.4f})")

## 6. 모델 평가 및 테스트

In [ ]:
# 추론 함수 정의
def generate_deobfuscated_text(obfuscated_text, max_length=256):
    """난독화된 텍스트를 입력받아 원본 텍스트 생성"""
    prompt = f"### 지시사항:\n다음 난독화된 한국어 텍스트를 원래 텍스트로 복원해주세요.\n\n난독화된 텍스트: {obfuscated_text}\n\n### 응답:\n"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 응답 부분만 추출
    if "### 응답:" in response:
        response = response.split("### 응답:")[1].strip()

    return response

# 평가 메트릭 함수 정의
def calculate_metrics(predictions, references):
    """BLEU, ROUGE 스코어 계산"""
    from evaluate import load
    
    # 메트릭 로드
    bleu = load("bleu")
    rouge = load("rouge")
    
    # BLEU 계산
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

    # ROUGE 계산
    rouge_score = rouge.compute(predictions=predictions, references=references)

    return {
        'bleu': bleu_score['bleu'],
        'rouge1': rouge_score['rouge1'],
        'rouge2': rouge_score['rouge2'],
        'rougeL': rouge_score['rougeL']
    }

print("추론 및 평가 함수 정의 완료")

In [ ]:
# 각 배치 사이즈별 모델 테스트
print("=== 배치 사이즈별 모델 성능 테스트 ===")

test_samples = val_df.sample(n=5, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n{'='*60}")
    print(f"배치 사이즈 {batch_size} 모델 테스트")
    print(f"{'='*60}")
    
    # 해당 배치 사이즈의 모델 로드
    model_path = RESULTS[batch_size]['model_path']
    
    # 베이스 모델 로드
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # LoRA 어댑터 적용
    from peft import PeftModel
    model = PeftModel.from_pretrained(base_model, model_path)
    
    # 테스트 샘플 평가
    for idx, row in test_samples.iterrows():
        obfuscated = row['input'].split("난독화된 텍스트: ")[1]
        original = row['output']
        predicted = generate_deobfuscated_text(obfuscated)

        print(f"\n[샘플 {idx}] 카테고리: {row['category']}")
        print(f"난독화: {obfuscated}")
        print(f"원본: {original}")
        print(f"예측: {predicted}")
        print("-" * 40)
    
    # 메모리 정리
    del model, base_model
    torch.cuda.empty_cache()

In [ ]:
# 배치 사이즈별 정량적 평가
batch_metrics = {}

print("=== 배치 사이즈별 정량적 평가 ===")

# 평가용 샘플 (50개로 축소)
eval_samples = val_df.sample(n=50, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n배치 사이즈 {batch_size} 평가 중...")
    
    # 모델 로드
    model_path = RESULTS[batch_size]['model_path']
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    model = PeftModel.from_pretrained(base_model, model_path)
    
    predictions = []
    references = []
    
    for idx, row in eval_samples.iterrows():
        obfuscated = row['input'].split("난독화된 텍스트: ")[1]
        original = row['output']
        predicted = generate_deobfuscated_text(obfuscated)
        
        predictions.append(predicted)
        references.append(original)
    
    # 메트릭 계산
    metrics = calculate_metrics(predictions, references)
    batch_metrics[batch_size] = metrics
    
    print(f"BLEU Score: {metrics['bleu']:.4f}")
    print(f"ROUGE-1: {metrics['rouge1']:.4f}")
    print(f"ROUGE-2: {metrics['rouge2']:.4f}")
    print(f"ROUGE-L: {metrics['rougeL']:.4f}")
    
    # 메모리 정리
    del model, base_model
    torch.cuda.empty_cache()

# 결과 비교표
print("\n=== 배치 사이즈별 성능 비교 ===")
print("┌─────────────┬──────────┬──────────┬──────────┬──────────┐")
print("│ Batch Size  │   BLEU   │ ROUGE-1  │ ROUGE-2  │ ROUGE-L  │")
print("├─────────────┼──────────┼──────────┼──────────┼──────────┤")

for batch_size in BATCH_SIZES:
    metrics = batch_metrics[batch_size]
    print(f"│     {batch_size:2d}      │  {metrics['bleu']:6.4f}  │  {metrics['rouge1']:6.4f}  │  {metrics['rouge2']:6.4f}  │  {metrics['rougeL']:6.4f}  │")

print("└─────────────┴──────────┴──────────┴──────────┴──────────┘")

# 최고 성능 배치 사이즈
best_bleu_batch = max(batch_metrics.keys(), key=lambda x: batch_metrics[x]['bleu'])
best_rouge_batch = max(batch_metrics.keys(), key=lambda x: batch_metrics[x]['rougeL'])

print(f"\n🏆 최고 BLEU 성능: 배치 사이즈 {best_bleu_batch} (BLEU: {batch_metrics[best_bleu_batch]['bleu']:.4f})")
print(f"🏆 최고 ROUGE-L 성능: 배치 사이즈 {best_rouge_batch} (ROUGE-L: {batch_metrics[best_rouge_batch]['rougeL']:.4f})")

## 6. 모델 평가 및 테스트

In [ ]:
# 추론 함수 정의
def generate_deobfuscated_text(obfuscated_text, max_length=256):
    """난독화된 텍스트를 입력받아 원본 텍스트 생성"""
    prompt = f"### 지시사항:\n다음 난독화된 한국어 텍스트를 원래 텍스트로 복원해주세요.\n\n난독화된 텍스트: {obfuscated_text}\n\n### 응답:\n"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 응답 부분만 추출
    if "### 응답:" in response:
        response = response.split("### 응답:")[1].strip()

    return response

# 평가 메트릭 함수 정의
def calculate_metrics(predictions, references):
    """BLEU, ROUGE 스코어 계산"""
    from evaluate import load
    
    # 메트릭 로드
    bleu = load("bleu")
    rouge = load("rouge")
    
    # BLEU 계산
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

    # ROUGE 계산
    rouge_score = rouge.compute(predictions=predictions, references=references)

    return {
        'bleu': bleu_score['bleu'],
        'rouge1': rouge_score['rouge1'],
        'rouge2': rouge_score['rouge2'],
        'rougeL': rouge_score['rougeL']
    }

print("추론 및 평가 함수 정의 완료")

In [ ]:
# 각 배치 사이즈별 모델 테스트
print("=== 배치 사이즈별 모델 성능 테스트 ===")

test_samples = val_df.sample(n=5, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n{'='*60}")
    print(f"배치 사이즈 {batch_size} 모델 테스트")
    print(f"{'='*60}")
    
    # 해당 배치 사이즈의 모델 로드
    model_path = RESULTS[batch_size]['model_path']
    
    # 베이스 모델 로드
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # LoRA 어댑터 적용
    from peft import PeftModel
    model = PeftModel.from_pretrained(base_model, model_path)
    
    # 테스트 샘플 평가
    for idx, row in test_samples.iterrows():
        obfuscated = row['input'].split("난독화된 텍스트: ")[1]
        original = row['output']
        predicted = generate_deobfuscated_text(obfuscated)

        print(f"\n[샘플 {idx}] 카테고리: {row['category']}")
        print(f"난독화: {obfuscated}")
        print(f"원본: {original}")
        print(f"예측: {predicted}")
        print("-" * 40)
    
    # 메모리 정리
    del model, base_model
    torch.cuda.empty_cache()

In [ ]:
# 배치 사이즈별 정량적 평가
batch_metrics = {}

print("=== 배치 사이즈별 정량적 평가 ===")

# 평가용 샘플 (50개로 축소)
eval_samples = val_df.sample(n=50, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n배치 사이즈 {batch_size} 평가 중...")
    
    # 모델 로드
    model_path = RESULTS[batch_size]['model_path']
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    model = PeftModel.from_pretrained(base_model, model_path)
    
    predictions = []
    references = []
    
    for idx, row in eval_samples.iterrows():
        obfuscated = row['input'].split("난독화된 텍스트: ")[1]
        original = row['output']
        predicted = generate_deobfuscated_text(obfuscated)
        
        predictions.append(predicted)
        references.append(original)
    
    # 메트릭 계산
    metrics = calculate_metrics(predictions, references)
    batch_metrics[batch_size] = metrics
    
    print(f"BLEU Score: {metrics['bleu']:.4f}")
    print(f"ROUGE-1: {metrics['rouge1']:.4f}")
    print(f"ROUGE-2: {metrics['rouge2']:.4f}")
    print(f"ROUGE-L: {metrics['rougeL']:.4f}")
    
    # 메모리 정리
    del model, base_model
    torch.cuda.empty_cache()

# 결과 비교표
print("\n=== 배치 사이즈별 성능 비교 ===")
print("┌─────────────┬──────────┬──────────┬──────────┬──────────┐")
print("│ Batch Size  │   BLEU   │ ROUGE-1  │ ROUGE-2  │ ROUGE-L  │")
print("├─────────────┼──────────┼──────────┼──────────┼──────────┤")

for batch_size in BATCH_SIZES:
    metrics = batch_metrics[batch_size]
    print(f"│     {batch_size:2d}      │  {metrics['bleu']:6.4f}  │  {metrics['rouge1']:6.4f}  │  {metrics['rouge2']:6.4f}  │  {metrics['rougeL']:6.4f}  │")

print("└─────────────┴──────────┴──────────┴──────────┴──────────┘")

# 최고 성능 배치 사이즈
best_bleu_batch = max(batch_metrics.keys(), key=lambda x: batch_metrics[x]['bleu'])
best_rouge_batch = max(batch_metrics.keys(), key=lambda x: batch_metrics[x]['rougeL'])

print(f"\n🏆 최고 BLEU 성능: 배치 사이즈 {best_bleu_batch} (BLEU: {batch_metrics[best_bleu_batch]['bleu']:.4f})")
print(f"🏆 최고 ROUGE-L 성능: 배치 사이즈 {best_rouge_batch} (ROUGE-L: {batch_metrics[best_rouge_batch]['rougeL']:.4f})")

## 6. 모델 평가 및 테스트

In [ ]:
# 추론 함수 정의
def generate_deobfuscated_text(obfuscated_text, max_length=256):
    """난독화된 텍스트를 입력받아 원본 텍스트 생성"""
    prompt = f"### 지시사항:\n다음 난독화된 한국어 텍스트를 원래 텍스트로 복원해주세요.\n\n난독화된 텍스트: {obfuscated_text}\n\n### 응답:\n"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 응답 부분만 추출
    if "### 응답:" in response:
        response = response.split("### 응답:")[1].strip()

    return response

# 평가 메트릭 함수 정의
def calculate_metrics(predictions, references):
    """BLEU, ROUGE 스코어 계산"""
    from evaluate import load
    
    # 메트릭 로드
    bleu = load("bleu")
    rouge = load("rouge")
    
    # BLEU 계산
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

    # ROUGE 계산
    rouge_score = rouge.compute(predictions=predictions, references=references)

    return {
        'bleu': bleu_score['bleu'],
        'rouge1': rouge_score['rouge1'],
        'rouge2': rouge_score['rouge2'],
        'rougeL': rouge_score['rougeL']
    }

print("추론 및 평가 함수 정의 완료")

In [ ]:
# 각 배치 사이즈별 모델 테스트
print("=== 배치 사이즈별 모델 성능 테스트 ===")

test_samples = val_df.sample(n=5, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n{'='*60}")
    print(f"배치 사이즈 {batch_size} 모델 테스트")
    print(f"{'='*60}")
    
    # 해당 배치 사이즈의 모델 로드
    model_path = RESULTS[batch_size]['model_path']
    
    # 베이스 모델 로드
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # LoRA 어댑터 적용
    from peft import PeftModel
    model = PeftModel.from_pretrained(base_model, model_path)
    
    # 테스트 샘플 평가
    for idx, row in test_samples.iterrows():
        obfuscated = row['input'].split("난독화된 텍스트: ")[1]
        original = row['output']
        predicted = generate_deobfuscated_text(obfuscated)

        print(f"\n[샘플 {idx}] 카테고리: {row['category']}")
        print(f"난독화: {obfuscated}")
        print(f"원본: {original}")
        print(f"예측: {predicted}")
        print("-" * 40)
    
    # 메모리 정리
    del model, base_model
    torch.cuda.empty_cache()

In [ ]:
# 배치 사이즈별 정량적 평가
batch_metrics = {}

print("=== 배치 사이즈별 정량적 평가 ===")

# 평가용 샘플 (50개로 축소)
eval_samples = val_df.sample(n=50, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n배치 사이즈 {batch_size} 평가 중...")
    
    # 모델 로드
    model_path = RESULTS[batch_size]['model_path']
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    model = PeftModel.from_pretrained(base_model, model_path)
    
    predictions = []
    references = []
    
    for idx, row in eval_samples.iterrows():
        obfuscated = row['input'].split("난독화된 텍스트: ")[1]
        original = row['output']
        predicted = generate_deobfuscated_text(obfuscated)
        
        predictions.append(predicted)
        references.append(original)
    
    # 메트릭 계산
    metrics = calculate_metrics(predictions, references)
    batch_metrics[batch_size] = metrics
    
    print(f"BLEU Score: {metrics['bleu']:.4f}")
    print(f"ROUGE-1: {metrics['rouge1']:.4f}")
    print(f"ROUGE-2: {metrics['rouge2']:.4f}")
    print(f"ROUGE-L: {metrics['rougeL']:.4f}")
    
    # 메모리 정리
    del model, base_model
    torch.cuda.empty_cache()

# 결과 비교표
print("\n=== 배치 사이즈별 성능 비교 ===")
print("┌─────────────┬──────────┬──────────┬──────────┬──────────┐")
print("│ Batch Size  │   BLEU   │ ROUGE-1  │ ROUGE-2  │ ROUGE-L  │")
print("├─────────────┼──────────┼──────────┼──────────┼──────────┤")

for batch_size in BATCH_SIZES:
    metrics = batch_metrics[batch_size]
    print(f"│     {batch_size:2d}      │  {metrics['bleu']:6.4f}  │  {metrics['rouge1']:6.4f}  │  {metrics['rouge2']:6.4f}  │  {metrics['rougeL']:6.4f}  │")

print("└─────────────┴──────────┴──────────┴──────────┴──────────┘")

# 최고 성능 배치 사이즈
best_bleu_batch = max(batch_metrics.keys(), key=lambda x: batch_metrics[x]['bleu'])
best_rouge_batch = max(batch_metrics.keys(), key=lambda x: batch_metrics[x]['rougeL'])

print(f"\n🏆 최고 BLEU 성능: 배치 사이즈 {best_bleu_batch} (BLEU: {batch_metrics[best_bleu_batch]['bleu']:.4f})")
print(f"🏆 최고 ROUGE-L 성능: 배치 사이즈 {best_rouge_batch} (ROUGE-L: {batch_metrics[best_rouge_batch]['rougeL']:.4f})")

## 6. 모델 평가 및 테스트

In [ ]:
# 추론 함수 정의
def generate_deobfuscated_text(obfuscated_text, max_length=256):
    """난독화된 텍스트를 입력받아 원본 텍스트 생성"""
    prompt = f"### 지시사항:\n다음 난독화된 한국어 텍스트를 원래 텍스트로 복원해주세요.\n\n난독화된 텍스트: {obfuscated_text}\n\n### 응답:\n"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 응답 부분만 추출
    if "### 응답:" in response:
        response = response.split("### 응답:")[1].strip()

    return response

# 평가 메트릭 함수 정의
def calculate_metrics(predictions, references):
    """BLEU, ROUGE 스코어 계산"""
    from evaluate import load
    
    # 메트릭 로드
    bleu = load("bleu")
    rouge = load("rouge")
    
    # BLEU 계산
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

    # ROUGE 계산
    rouge_score = rouge.compute(predictions=predictions, references=references)

    return {
        'bleu': bleu_score['bleu'],
        'rouge1': rouge_score['rouge1'],
        'rouge2': rouge_score['rouge2'],
        'rougeL': rouge_score['rougeL']
    }

print("추론 및 평가 함수 정의 완료")

In [ ]:
# 각 배치 사이즈별 모델 테스트
print("=== 배치 사이즈별 모델 성능 테스트 ===")

test_samples = val_df.sample(n=5, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n{'='*60}")
    print(f"배치 사이즈 {batch_size} 모델 테스트")
    print(f"{'='*60}")
    
    # 해당 배치 사이즈의 모델 로드
    model_path = RESULTS[batch_size]['model_path']
    
    # 베이스 모델 로드
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # LoRA 어댑터 적용
    from peft import PeftModel
    model = PeftModel.from_pretrained(base_model, model_path)
    
    # 테스트 샘플 평가
    for idx, row in test_samples.iterrows():
        obfuscated = row['input'].split("난독화된 텍스트: ")[1]
        original = row['output']
        predicted = generate_deobfuscated_text(obfuscated)

        print(f"\n[샘플 {idx}] 카테고리: {row['category']}")
        print(f"난독화: {obfuscated}")
        print(f"원본: {original}")
        print(f"예측: {predicted}")
        print("-" * 40)
    
    # 메모리 정리
    del model, base_model
    torch.cuda.empty_cache()

In [ ]:
# 배치 사이즈별 정량적 평가
batch_metrics = {}

print("=== 배치 사이즈별 정량적 평가 ===")

# 평가용 샘플 (50개로 축소)
eval_samples = val_df.sample(n=50, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n배치 사이즈 {batch_size} 평가 중...")
    
    # 모델 로드
    model_path = RESULTS[batch_size]['model_path']
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    model = PeftModel.from_pretrained(base_model, model_path)
    
    predictions = []
    references = []
    
    for idx, row in eval_samples.iterrows():
        obfuscated = row['input'].split("난독화된 텍스트: ")[1]
        original = row['output']
        predicted = generate_deobfuscated_text(obfuscated)
        
        predictions.append(predicted)
        references.append(original)
    
    # 메트릭 계산
    metrics = calculate_metrics(predictions, references)
    batch_metrics[batch_size] = metrics
    
    print(f"BLEU Score: {metrics['bleu']:.4f}")
    print(f"ROUGE-1: {metrics['rouge1']:.4f}")
    print(f"ROUGE-2: {metrics['rouge2']:.4f}")
    print(f"ROUGE-L: {metrics['rougeL']:.4f}")
    
    # 메모리 정리
    del model, base_model
    torch.cuda.empty_cache()

# 결과 비교표
print("\n=== 배치 사이즈별 성능 비교 ===")
print("┌─────────────┬──────────┬──────────┬──────────┬──────────┐")
print("│ Batch Size  │   BLEU   │ ROUGE-1  │ ROUGE-2  │ ROUGE-L  │")
print("├─────────────┼──────────┼──────────┼──────────┼──────────┤")

for batch_size in BATCH_SIZES:
    metrics = batch_metrics[batch_size]
    print(f"│     {batch_size:2d}      │  {metrics['bleu']:6.4f}  │  {metrics['rouge1']:6.4f}  │  {metrics['rouge2']:6.4f}  │  {metrics['rougeL']:6.4f}  │")

print("└─────────────┴──────────┴──────────┴──────────┴──────────┘")

# 최고 성능 배치 사이즈
best_bleu_batch = max(batch_metrics.keys(), key=lambda x: batch_metrics[x]['bleu'])
best_rouge_batch = max(batch_metrics.keys(), key=lambda x: batch_metrics[x]['rougeL'])

print(f"\n🏆 최고 BLEU 성능: 배치 사이즈 {best_bleu_batch} (BLEU: {batch_metrics[best_bleu_batch]['bleu']:.4f})")
print(f"🏆 최고 ROUGE-L 성능: 배치 사이즈 {best_rouge_batch} (ROUGE-L: {batch_metrics[best_rouge_batch]['rougeL']:.4f})")

## 6. 모델 평가 및 테스트

In [ ]:
# 추론 함수 정의
def generate_deobfuscated_text(obfuscated_text, max_length=256):
    """난독화된 텍스트를 입력받아 원본 텍스트 생성"""
    prompt = f"### 지시사항:\n다음 난독화된 한국어 텍스트를 원래 텍스트로 복원해주세요.\n\n난독화된 텍스트: {obfuscated_text}\n\n### 응답:\n"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 응답 부분만 추출
    if "### 응답:" in response:
        response = response.split("### 응답:")[1].strip()

    return response

# 평가 메트릭 함수 정의
def calculate_metrics(predictions, references):
    """BLEU, ROUGE 스코어 계산"""
    from evaluate import load
    
    # 메트릭 로드
    bleu = load("bleu")
    rouge = load("rouge")
    
    # BLEU 계산
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

    # ROUGE 계산
    rouge_score = rouge.compute(predictions=predictions, references=references)

    return {
        'bleu': bleu_score['bleu'],
        'rouge1': rouge_score['rouge1'],
        'rouge2': rouge_score['rouge2'],
        'rougeL': rouge_score['rougeL']
    }

print("추론 및 평가 함수 정의 완료")

In [ ]:
# 각 배치 사이즈별 모델 테스트
print("=== 배치 사이즈별 모델 성능 테스트 ===")

test_samples = val_df.sample(n=5, random_state=42)

for batch_size in BATCH_SIZES:
    print(f"\n{'='*60}")
    print(f"배치 사이